# Imports & Environment Setup

In [2]:
import os, random, math, numpy as np, pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from sklearn.metrics import classification_report, confusion_matrix

# Set Random Seeds

In [3]:
def seed_all(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_all(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Loading Dataset - Dataset Paths (Train / Valid / Test)

In [ ]:
DATA_ROOT = "/kaggle/input/70-dog-breedsimage-data-set"

train_dir = os.path.join(DATA_ROOT, "train")
val_dir   = os.path.join(DATA_ROOT, "valid")
test_dir  = os.path.join(DATA_ROOT, "test")

print("Train exists:", os.path.isdir(train_dir))
print("Valid exists:", os.path.isdir(val_dir))
print("Test exists :", os.path.isdir(test_dir))


# Number of Classes

In [ ]:
def count_images(root):
    classes = sorted([d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))])
    counts = []
    for c in classes:
        cpath = os.path.join(root, c)
        n = len([f for f in os.listdir(cpath) if f.lower().endswith((".jpg",".jpeg",".png"))])
        counts.append((c, n))
    df = pd.DataFrame(counts, columns=["class", "count"]).sort_values("count", ascending=False)
    return df, classes

train_df, classes = count_images(train_dir)
val_df, _ = count_images(val_dir)
test_df, _ = count_images(test_dir)

print("Num classes:", len(classes))
print("Train images:", train_df["count"].sum())
print("Val images  :", val_df["count"].sum())
print("Test images :", test_df["count"].sum())

train_df.head()


# Class Distribution: Count Images per Breed

In [ ]:
plt.figure(figsize=(12,4))
plt.hist(train_df["count"], bins=20)
plt.title("Train: Distribution of images per class")
plt.xlabel("Images per class")
plt.ylabel("Number of classes")
plt.show()

train_df.describe()


# Visualize Random Training Samples

In [ ]:
def show_samples(root, classes, n=12):
    plt.figure(figsize=(12,8))
    for i in range(n):
        c = random.choice(classes)
        cdir = os.path.join(root, c)
        img_name = random.choice(os.listdir(cdir))
        img_path = os.path.join(cdir, img_name)
        img = Image.open(img_path).convert("RGB")
        plt.subplot(3,4,i+1)
        plt.imshow(img)
        plt.title(c, fontsize=9)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

show_samples(train_dir, classes, n=12)


# Check Image Size Variations

In [ ]:
from collections import Counter

def sample_sizes(root, classes, k=300):
    sizes = []
    for _ in range(k):
        c = random.choice(classes)
        cdir = os.path.join(root, c)
        img_name = random.choice(os.listdir(cdir))
        img_path = os.path.join(cdir, img_name)
        img = Image.open(img_path).convert("RGB")
        sizes.append(img.size)
    return Counter(sizes).most_common(10)

sample_sizes(train_dir, classes, k=300)

# Compute Dataset Mean & Std for Normalization

In [ ]:
def compute_mean_std(dataset, num_batches=50, batch_size=64):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    mean = 0.
    var  = 0.
    n = 0
    for i, (x, _) in enumerate(loader):
        if i >= num_batches: break
        x = x.to(device)
        b = x.size(0)
        x = x.view(b, x.size(1), -1)
        mean += x.mean(dim=2).sum(dim=0)
        var  += x.var(dim=2, unbiased=False).sum(dim=0)
        n += b
    mean /= n
    var  /= n
    std = torch.sqrt(var)
    return mean.detach().cpu().numpy(), std.detach().cpu().numpy()

In [ ]:
stats_tf = transforms.Compose([transforms.ToTensor()])
tmp_ds = datasets.ImageFolder(train_dir, transform=stats_tf)
mean, std = compute_mean_std(tmp_ds)
mean, std

# Data Augmentation & Preprocessing Pipelines

In [ ]:
IMG_SIZE = 256
BATCH_SIZE = 64

train_tf = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.05, 0.05)),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.12), ratio=(0.3, 3.3)),
])

eval_tf = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

from torchvision.datasets import ImageFolder
import re

def canon(s: str) -> str:
    return re.sub(r"\s+", " ", s.strip()).lower()

class ImageFolderWithCanonicalMap(ImageFolder):
    def __init__(self, root, train_class_to_idx, transform=None):
        super().__init__(root=root, transform=transform)
        train_c2i = {canon(k): v for k, v in train_class_to_idx.items()}
        fixed_samples = []
        missing = set()
        for path, local_target in self.samples:
            class_name = self.classes[local_target]
            key = canon(class_name)
            if key not in train_c2i:
                missing.add(class_name)
                continue
            fixed_samples.append((path, train_c2i[key]))
        if missing:
            print("WARNING: unmatched classes:", list(sorted(missing))[:10], "...")
        self.samples = fixed_samples
        self.targets = [t for _, t in self.samples]

train_ds = ImageFolder(train_dir, transform=train_tf)
val_ds   = ImageFolderWithCanonicalMap(val_dir,  train_ds.class_to_idx, transform=eval_tf)
test_ds  = ImageFolderWithCanonicalMap(test_dir, train_ds.class_to_idx, transform=eval_tf)

num_classes = len(train_ds.classes)
print("Classes:", num_classes)
print("Val samples:", len(val_ds))
print("Test samples:", len(test_ds))

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)